In [2]:
# ================================================================
# 🪴 Medicinal Leaf Classification - Flask-Ready Model
# Classes: Mango, Neem, Guava, Lemon
# Authors: (Ayuen, Malith, Deng, Biar & Akot)
# ================================================================
# This notebook trains a leaf classifier in two stages:
#  1️⃣ Base CNN from scratch
#  2️⃣ Transfer Learning using EfficientNetB7
# The final model is saved for easy use in a Flask web app.
# ================================================================

# ------------------------------
# 1. IMPORT LIBRARIES
# ------------------------------
import os
import random
from glob import glob
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import cv2
import albumentations as A
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.applications.efficientnet import preprocess_input as effnet_preprocess
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

print('✅ TensorFlow version:', tf.__version__)


✅ TensorFlow version: 2.18.0


In [3]:
# ------------------------------
# 2. CONFIGURATION
# ------------------------------
# Adjust these paths for your environment (Kaggle or local)
DATA_DIR = '/kaggle/input/indian-medicinal-leaves-dataset/Indian Medicinal Leaves Image Datasets/Medicinal Leaf dataset'
WORK_DIR = '/kaggle/working/models'

# Create working directory if not exists
os.makedirs(WORK_DIR, exist_ok=True)

# Image and training parameters
IMG_SIZE = 224
BATCH_SIZE = 16
EPOCHS_BASE = 8
EPOCHS_TRANSFER = 10
RANDOM_SEED = 42

# Target classes (4 specific plants)
TARGET_CLASSES = ['Mango', 'Neem', 'Guava', 'Lemon']

# Set global seeds for reproducibility
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [4]:
# ------------------------------
# 3. BUILD METADATA FROM FOLDERS
# ------------------------------
def build_meta_from_folders(root_dir, extensions=('jpg', 'jpeg', 'png')):
    """
    Scans dataset folders and returns a DataFrame with image file paths and labels.
    """
    rows = []
    for cls in sorted(os.listdir(root_dir)):
        if cls not in TARGET_CLASSES:
            continue  # Ignore unwanted classes
        cls_path = os.path.join(root_dir, cls)
        if not os.path.isdir(cls_path):
            continue
        for ext in extensions:
            for fp in glob(os.path.join(cls_path, '**', f'*.{ext}'), recursive=True):
                rows.append({'file_path': fp, 'label': cls})
    return pd.DataFrame(rows)

meta = build_meta_from_folders(DATA_DIR)
if meta.empty:
    raise ValueError(f"No images found for classes {TARGET_CLASSES}")

print(f"📸 Total images found: {len(meta)}")

📸 Total images found: 486


In [5]:
# ------------------------------
# 4. SPLIT DATA INTO TRAIN/VAL/TEST
# ------------------------------
train_df, temp_df = train_test_split(meta, test_size=0.3, stratify=meta['label'], random_state=RANDOM_SEED)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=RANDOM_SEED)

print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

# Encode class labels as integers
class_names = sorted(train_df['label'].unique())
label2idx = {c: i for i, c in enumerate(class_names)}

# Save label mapping for Flask app
with open(os.path.join(WORK_DIR, "class_indices.json"), "w") as f:
    json.dump({"class_names": class_names, "label2idx": label2idx}, f, indent=4)
print("✅ Saved class mapping → class_indices.json")

Train: 340, Val: 73, Test: 73
✅ Saved class mapping → class_indices.json


In [6]:
# ------------------------------
# 5. DATA AUGMENTATION PIPELINES
# ------------------------------
train_transform = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=20, p=0.5),
])

val_transform = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE)
])

def _read_image(path):
    """
    Reads and converts image to RGB.
    """
    img = cv2.imread(path.decode('utf-8'))
    if img is None:
        return np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def augment_image(path, label, transform):
    """
    Applies augmentation (Albumentations) and scales image to [0,1].
    """
    img = _read_image(path)
    augmented = transform(image=img)
    img = augmented['image'].astype('float32') / 255.0
    return img, label

def tf_augment(path, label, train=True):
    """
    Wraps Python image augmentations for TensorFlow Dataset.
    """
    fn = lambda p, l: augment_image(p, l, train_transform if train else val_transform)
    img, lbl = tf.numpy_function(fn, [path, label], [tf.float32, tf.int64])
    img.set_shape((IMG_SIZE, IMG_SIZE, 3))
    lbl.set_shape(())
    return img, lbl

def df_to_dataset(df, batch_size=BATCH_SIZE, shuffle=True, train=True):
    """
    Converts DataFrame into tf.data.Dataset with augmentations.
    """
    paths = df['file_path'].astype(str).values
    labels = df['label'].map(label2idx).astype(np.int64).values
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(paths), seed=RANDOM_SEED)
    ds = ds.map(lambda p, l: tf_augment(p, l, train=train), num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

# Create datasets
train_dataset = df_to_dataset(train_df, train=True)
val_dataset = df_to_dataset(val_df, train=False)
test_dataset = df_to_dataset(test_df, train=False)

/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
I0000 00:00:1762108723.669914      37 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [7]:
# ------------------------------
# 6. BASE CNN MODEL (Stage 1)
# ------------------------------
def build_base_model(input_shape=(IMG_SIZE, IMG_SIZE, 3), n_classes=len(class_names)):
    """
    Simple CNN baseline for initial training.
    """
    inputs = keras.Input(shape=input_shape)
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
    x = layers.MaxPool2D()(x)
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.MaxPool2D()(x)
    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(n_classes, activation='softmax')(x)
    return keras.Model(inputs, outputs)

base_model = build_base_model()
base_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

callbacks_base = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint(os.path.join(WORK_DIR, 'base_model.keras'), save_best_only=True, monitor='val_loss')
]

print("\n🚀 Training Base CNN...\n")
history_base = base_model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS_BASE, callbacks=callbacks_base)


🚀 Training Base CNN...

Epoch 1/8


I0000 00:00:1762108736.103825     103 service.cc:148] XLA service 0x78aa94087840 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1762108736.104648     103 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1762108736.463684     103 cuda_dnn.cc:529] Loaded cuDNN version 90300


 6/22 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.2816 - loss: 1.3972

I0000 00:00:1762108739.380979     103 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 194ms/step - accuracy: 0.2803 - loss: 1.3899 - val_accuracy: 0.2740 - val_loss: 1.3794
Epoch 2/8
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.3240 - loss: 1.3714 - val_accuracy: 0.3014 - val_loss: 1.3640
Epoch 3/8
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.2762 - loss: 1.3757 - val_accuracy: 0.2740 - val_loss: 1.3653
Epoch 4/8
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.3284 - loss: 1.3679 - val_accuracy: 0.2877 - val_loss: 1.3498
Epoch 5/8
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.3741 - loss: 1.3368 - val_accuracy: 0.4384 - val_loss: 1.2888
Epoch 6/8
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.3996 - loss: 1.2814 - val_accuracy: 0.3425 - val_loss: 1.2807
Epoch 7/8
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.4105 - loss: 1.2736 - val_accuracy: 0.5205 - val_loss: 1.2069
Epoch 8/8
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.4147 - loss: 1.2284 - val_accuracy: 0.4521 - val_loss: 1.1545


In [12]:
# ------------------------------
# 7. EFFICIENTNETB7 TRANSFER LEARNING (Stage 2)
# ------------------------------
def build_efficientnet_model(n_classes=len(class_names)):
    """
    Builds EfficientNetB7-based model for transfer learning.
    """
    base = EfficientNetB7(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3), pooling='avg')
    base.trainable = False  # Freeze base initially

    inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = effnet_preprocess(inputs * 255.0)
    x = base(x, training=False)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(n_classes, activation='softmax')(x)
    return keras.Model(inputs, outputs)

effnet_model = build_efficientnet_model()
effnet_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

callbacks_effnet = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint(os.path.join(WORK_DIR, 'efficientnet_stage1.keras'), save_best_only=True, monitor='val_loss')
]

print("\n🌱 Training EfficientNetB7 (Frozen base)...\n")
history_effnet = effnet_model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS_TRANSFER, callbacks=callbacks_effnet)

258076736/258076736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step

🌱 Training EfficientNetB7 (Frozen base)...

Epoch 1/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 143s 3s/step - accuracy: 0.3954 - loss: 1.3273 - val_accuracy: 0.7808 - val_loss: 0.8109
Epoch 2/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 258ms/step - accuracy: 0.7572 - loss: 0.8269 - val_accuracy: 0.8493 - val_loss: 0.6081
Epoch 3/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 300ms/step - accuracy: 0.7737 - loss: 0.6640 - val_accuracy: 0.8493 - val_loss: 0.5029
Epoch 4/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 257ms/step - accuracy: 0.8744 - loss: 0.4944 - val_accuracy: 0.8630 - val_loss: 0.4463
Epoch 5/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 258ms/step - accuracy: 0.8803 - loss: 0.4207 - val_accuracy: 0.8630 - val_loss: 0.4160
Epoch 6/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 256ms/step - accuracy: 0.8807 - loss: 0.4434 - val_accuracy: 0.9041 - val_loss: 0.3807
Epoch 7/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 257ms/step - accuracy: 0.9086 - loss: 0.3263 - val_accuracy: 0.8767 - val_loss: 0.3521
Epoch 8

In [13]:
# ------------------------------
# 8. FINE-TUNING EFFICIENTNETB7
# ------------------------------
# Unfreeze base model for fine-tuning
effnet_model.get_layer(index=1).trainable = True
effnet_model.compile(optimizer=keras.optimizers.Adam(1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

callbacks_finetune = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ModelCheckpoint(os.path.join(WORK_DIR, 'efficientnet_finetuned.keras'), save_best_only=True, monitor='val_loss')
]

print("\n🔧 Fine-tuning EfficientNetB7...\n")
history_finetune = effnet_model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS_TRANSFER, callbacks=callbacks_finetune)


🔧 Fine-tuning EfficientNetB7...

Epoch 1/10


E0000 00:00:1762109639.412407     105 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1762109639.608624     105 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1762109639.998312     105 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1762109640.200974     105 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1762109640.701092     105 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:0

21/22 ━━━━━━━━━━━━━━━━━━━━ 0s 372ms/step - accuracy: 0.6736 - loss: 0.8573

E0000 00:00:1762109741.160307     106 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1762109741.359651     106 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1762109741.713771     106 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1762109741.932991     106 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1762109742.515294     106 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:0

22/22 ━━━━━━━━━━━━━━━━━━━━ 404s 6s/step - accuracy: 0.6738 - loss: 0.8557 - val_accuracy: 0.9041 - val_loss: 0.3090
Epoch 2/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 9s 387ms/step - accuracy: 0.7340 - loss: 0.7549 - val_accuracy: 0.8904 - val_loss: 0.3169
Epoch 3/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 384ms/step - accuracy: 0.7685 - loss: 0.6436 - val_accuracy: 0.8630 - val_loss: 0.3407
Epoch 4/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 9s 385ms/step - accuracy: 0.8382 - loss: 0.5285 - val_accuracy: 0.8356 - val_loss: 0.3610


In [15]:
# ------------------------------
# 9. SAVE FINAL MODEL FOR FLASK
# ------------------------------
final_model_path = os.path.join(WORK_DIR, 'model_final_flask.h5')
effnet_model.save(final_model_path)
print("✅ Final model saved for Flask:", final_model_path)

✅ Final model saved for Flask: /kaggle/working/models/model_final_flask.h5


In [16]:
# ------------------------------
# 10. EVALUATION
# ------------------------------
y_true, y_pred = [], []
for imgs, lbls in test_dataset:
    preds = effnet_model.predict(imgs)
    y_true.extend(lbls.numpy().tolist())
    y_pred.extend(np.argmax(preds, axis=1).tolist())

print("\n📊 Classification Report:\n")
print(classification_report(y_true, y_pred, target_names=class_names))

1/1 ━━━━━━━━━━━━━━━━━━━━ 17s 17s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 20s 20s/step

📊 Classification Report:

              precision    recall  f1-score   support

       Guava       0.85      0.89      0.87        19
       Lemon       1.00      0.89      0.94        19
       Mango       0.82      0.93      0.87        15
        Neem       1.00      0.95      0.97        20

    accuracy                           0.92        73
   macro avg       0.92      0.92      0.92        73
weighted avg       0.92      0.92      0.92        73

